# Using Reinforcement Learning to Play a Fishing Game
*By Keegan Kochis and Landon Zweigle*

# Introduction
---
Ever since we have gotten into programming we have had a desire to do reinforcement learning, and this term project seemed like a perfect excuse to finally do it. When it came to deciding what to do RL on, we decided to use a [JavaFX](https://openjfx.io/) game Landon made a few years ago; a very simple fishing minigame where the goal is to put a bobber over a fish for a certain amount of time. This game had everything we were looking for; an evironment which is rather simple, but not too simple; and an easy way to communicate with it.



The game as described earlier is a simple fishing minigame. It is actually a clone of minigame in a very popular indie game [Stardew Valley](https://www.stardewvalley.net/). Here is an indepth [description](https://stardewvalleywiki.com/Fishing#Overview_.26_Controls) of the fishing minigame. The image below is a screenshot from Stardew Valley's fishing minigame.

![The game](https://www.carlsguides.com/stardewvalley/pictures/fishing/catch-fish.jpg)


The mini-game is a simple objective of keeping a capture area (aka "the bobber") behind a randomly moving capture-point (aka "the fish") for long enough to win the game. In the image, the capture-point is the fish and the area you move (the bobber) is the green bar. You can change the velocity of the bobber by accelerating it by either holding space or releasing space and letting gravity pull it down. The capture time left is indicated by the large green bar on the left, the "capture bar". The goal is to raise the capture bar to the top of the mini-game window. If the capture bar decreases to the bottom, the fish will escape and you will lose. In the original game each time the minigame starts the fish's difficulty is randomized. However for our purposes the difficulty will be constant.




Because the game was built in Java and our RL algorithm would be implemented in Python, we needed a way to communicate between Python and Java. For this we decided to use socket programming. Further, because of the random movement of the fish, we knew we had to use a neural network to approximate the Q-table.

# Methods
---
For our solution we of course needed a nueral network class. For this, we used some code provided to us in the course, a nueral network class and it's sibling optimizer class. We regularly visited lectures (lectures [17](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~cs445/notebooks/17%20Reinforcement%20Learning%20with%20Neural%20Network%20as%20Q%20Function.ipynb), [18](https://nbviewer.jupyter.org/url/www.cs.colostate.edu/~cs445/notebooks/18.1%20Reinforcement%20Learning%20to%20Control%20a%20Marble.ipynb)) for information regarding reinforcement learning algorithms, and a bit of code. Everything else was written by us.

## Steps taken
#### Java-Python Communications
Because Landon wrote the game, he implemented the communication between Java and Python. There were several options he could have gone with but he ended up writting a simple java socket server which python would recieve the games state from, and map actions to, all while keeping the game loop synced to the RL algorithm loops.

#### Convenient RL Neural Network
Keegan integrated the neural network class with a few modifications. He implemented a trivial `EpsilonGreedy` method which had access to the nueral networks `self` variable. He also implemented a template `getReinforcement` method which would be later greatly tuned to increase algorithm performance.

#### Main loop
Landon worked on the "main loop" of our program. The main loop deals with the main processes in the RL algorithm; getting the data for each trial, training the model, saving the results, etc..

#### Secondary Code
For this project to work properly a lot of secondary code was neccessary. Such functions that were neccesary include a way of easily creating a csv of experiments to be run, and a way of executing experiments. At this point in development, we both contributed to this code.

#### Fine tunning
To experiment with producing a functional model both Landon and Keegan would frequently adjust what Java reports as the state, and what the reinforcement/reward is defined as.


## The Main Loop.
For our implementation we had two proccesses communicating synchronously. It took considerable effort to make Java work syncronously with Python. In this section we will discuss the two proccess.

### The java code:
In order to get our stand alone Java game to work with Python Landon needed to make several changes. To begin with, there were several bugs in the game that made the fish inconsitent. To be specific, much of the physics did not have a take into account frame rate. This caused the fish to zoom around (at the speed of sound) at very high framerates. We needed to get fix this so the physics would be consistant at very low framerates. Next, we needed a way of syncing the frame rate up with Python. To do this, we simply added a simple condition check at the begining of the frame loop in Java which would only continue with the rest of the code if Python gave it a singal to do so. After this, we needed a way of easily communicating to Python what the current state of the game was. To do this we simply added a Java method which would grab all the information from the game, and convert it into a string, where it would finally be sent to Python on request. 

Now that this was complete, we could implement the Java side of the main loop:

1. The loop conditional recieves an integer from Python, and if it is not zero the loop body will be executed. Otherwise the loop ends and the game stops. This integer is how Python reports the state of the loops (not to be confused with the state of the game!). This can tell Java to either quit the loop or start a new game (described below).

3. Inside the loop, Java uses the same integer (loop-state) as used in the loop conditional to check an if statement. If the condition is true Java resets the game to a random state which is immediatly reported to Python. This code is used in the case of a new Trial to get the initial state.

4.  Outside the if statement, Java recieves an integer which will relate to the action Python desires to take. Java will then perform the requested action and process the next frame.

5. Next Java will wait untill the last requested frame has fully been rendered/proccessed.

6. After the frame has been rendered, Java will send an integer to Python reporting that the frame has successfuly been rendered. This is only required so Python can verify that Java is still running properly.

7. Next, Java will grab the current state, convert it to a string, and then send it to Python to proccess. After this step Java return to the loop-conditional.

### The Python Code:
The Python code that deals with communications is identical to Java, and as such will be simplified in this section.

1. The loop conditional for python checks that the current frame is less than the total amount of frame to be proccessed (`FramePerTrial` * `nTrials`).

2. Next, Python reports to Java the state of the loop.

3. If the state of the loop is to start a new trial, Python grabs a new state from Java as the initial state and then uses the Epsilon greedy function to generate an initial action.

4. Afterward, Python will report the desired action to Java. If Python did not execute the last described step (optional step 3), it will instead use the saved action in a following step.

5. Next Python will recieve an int from Java checking it's health, and raise an error if the int is not expected.

6. After Python checks the health of Java, Python will recieve the game state from Java in the form of a string. Python will then proccess this String to get a list of integers.

7. Given the current state, Python will get the reinforcement and decide the next action to take using epsilon greedy.

8. With the new state-action pair, Python will put these values into corresponding array which will later be used for training.

9. Python now checks whether or not to end this trial and start a new one (currentFrame % framesPerTrial==0). If a new trial needs to be started, Python will do several things:
    1. First Python will proccess the data created over repeated steps of step 8.
    2. Next, using the proccessed data, Python will train the neural network with the proccessed data.
    3. After training, Python will decay Epsilon to make random movements less likely.
    
10. If there are more trials to be ran Python will continue looping, otherwise Python will:
    1. Tell Java to terminate
    2. create several output files and proceed to exit.


## Reinforcement: A cost.

The backbone of reinforcement learning is assining a value to certain state. Some states are more desirable than others, and as such you need a way of telling the Q-table this. In our case, it is quite simple; we want the bobber to be as close to the fish as possible. Furthermore, because the fish moves randomly, we would like the bobbers velocity to match that of the fish. In essence, we are trying to **minimize** the distance between the bobber and the fish, thus our reinforcement is a **cost** we would like to minimize. Considering these simple facts, this is how we came up with our reinforcement:

Because the bobber and the fish both have a veloctity, and thus a position, the reinforcement is a little more complex than just how far the bobber is from the fish. Throughout development we kept an ideaology in mind such that there are technically 4 types of states:
* `Best Colliding`: The bobber is behind the fish and their velocities are in the same direction (equal normalized velocity).

* `Simple Colliding`: The bobber is behind the fish, but they have differing normalized velocities.

* `Seperate`: The bobber is not behind the fish, but they have equivalent normalized velocities.

* `Poor Seperate`: The bobber is not behind the fish and their normalized velocites are inequivalant.

In general we knew we wanted to minimize the reinforcement, and as such we decided that the reinforcement each state was as such: `Best Colliding` < `Simple Colliding` < `Seperate` < `Poor Seperate`.

We first began with a simple reinforcement based off of this ideaoligy that follows the above definition. `Best Colliding` returned 0, `Simple Colliding` returned 1, `Seperate` returned 2, `Poor Seperate` returned 3.

After discussing our problem we decided that our reinforcement should also encode the fish-bobber distance. To do this we realized that we couldn't just return the distance between the two. For instance, we considered an example where the fish and bobber are not colliding and moving in the same direction. If the fish's position is greater than the bobber's position, and their velocities are upward, then the subsequent state is `Seperate`, but if their velocites are downward, then the subsequent state is `Poor Seperate`. However, if we just return the distance this ideaology would be lost. Because of this we only encode their distance when the state is `Seperate`. This introduced another problem, `Seperate`'s reinforcement value according to the ideaology should be between 1 and 3, whereas the distance could be anywhere from 0 to 1000. This meant we needed to have a standardized distance between 0 and 1, so we simply take the percentage of the distance over the total possible distance (distance/max_range). This didn't entirly solve our problem though, as the new reinforcement for `Seperate` would be between 0 and 1, when it should be between 1 and 3, so we simply added 1 to the percentage. This gave us a reinforcement function defined as:

    if `Best Colliding`:
        return 0
    elif `Simple Colliding`:
        return 1
    elif `Seperate`:
        return (distance/max_range) + 1
    elif `Poor Seperate`:
        return 3


## Training

For training we made two flexible sccripts; `creatExperiments` and `run`. `createExperiments` simply creates a csv with several experiments which will then be parsed in `run` and executed. `run` as mentioned before parses each experiment, where for every experiment it will launch an instance of the java game, and then run our main reinforcement script `AutoFish` passing in the training arguments. After each experiment `AutoFish` creates a directory with 4 files in it; `ActionState.csv`, `DQN.dump`, `meanReinByTrial.png` and `results.csv`.

* `ActionState.csv` is a record of the last-n state-action pairs. This is useful for knowing what action is made in several actions.
* `DQN.dump` is the trained Deep-Q-Network (DQN) which was pickled.
* `meanReinByTrial.png` is a graph displaying the mean reinforcement per trial. There is also a 20-trial average overlayed in orange. An example of the graph can be found below.
* `results.csv` displays the parameters for the experiment (I.E. network architecture, number of frames per trial, etc).

We will discuss further in the results exactly what we ran, but over the course of 3 weeks we ran around 100 experiments using these classes.

![fish](https://i.imgur.com/S0nNkDT.png)

# Results
----------

In [3]:
import io
from nbformat import current
import glob
name="Kochis-Zweigle"
nbfile = glob.glob(name+'.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Kochis-Zweigle.ipynb is 2153
